# **LSTM**

In [6]:
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt


In [7]:
# Create a simple sine wave sequence for training
def generate_data(seq_length):
    x = np.linspace(0, 2 * np.pi, seq_length)
    data = np.sin(x)
    return data

sequence_length = 50
data = generate_data(sequence_length + 1)

# Prepare input and target
x = torch.tensor(data[:-1], dtype=torch.float32).reshape(1, sequence_length, 1)
y = torch.tensor(data[1:], dtype=torch.float32).reshape(1, sequence_length, 1)


In [8]:
class LSTMModel(nn.Module):
    def __init__(self, input_size=1, hidden_size=50, num_layers=1, output_size=1):
        super(LSTMModel, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers

        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size)

        out, _ = self.lstm(x, (h0, c0))
        out = self.fc(out)
        return out


In [9]:
model = LSTMModel()
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

epochs = 200
for epoch in range(epochs):
    model.train()
    output = model(x)
    loss = criterion(output, y)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if (epoch + 1) % 20 == 0:
        print(f'Epoch [{epoch+1}/{epochs}], Loss: {loss.item():.4f}')


Epoch [20/200], Loss: 0.0046
Epoch [40/200], Loss: 0.0008
Epoch [60/200], Loss: 0.0001
Epoch [80/200], Loss: 0.0000
Epoch [100/200], Loss: 0.0000
Epoch [120/200], Loss: 0.0000
Epoch [140/200], Loss: 0.0000
Epoch [160/200], Loss: 0.0000
Epoch [180/200], Loss: 0.0000
Epoch [200/200], Loss: 0.0000


# **RNN**

In [11]:
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt


In [12]:
def generate_data(seq_length):
    x = np.linspace(0, 2 * np.pi, seq_length)
    data = np.sin(x)
    return data

sequence_length = 50
data = generate_data(sequence_length + 1)

# Prepare input and target tensors
x = torch.tensor(data[:-1], dtype=torch.float32).reshape(1, sequence_length, 1)  # shape: (batch, seq_len, input_size)
y = torch.tensor(data[1:], dtype=torch.float32).reshape(1, sequence_length, 1)  # shifted target


In [13]:
class RNNModel(nn.Module):
    def __init__(self, input_size=1, hidden_size=50, num_layers=1, output_size=1):
        super(RNNModel, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers

        self.rnn = nn.RNN(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size)

        out, _ = self.rnn(x, h0)
        out = self.fc(out)
        return out


In [14]:
model = RNNModel()
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

epochs = 200
for epoch in range(epochs):
    model.train()
    output = model(x)
    loss = criterion(output, y)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if (epoch + 1) % 20 == 0:
        print(f"Epoch [{epoch+1}/{epochs}], Loss: {loss.item():.4f}")


Epoch [20/200], Loss: 0.0028
Epoch [40/200], Loss: 0.0005
Epoch [60/200], Loss: 0.0001
Epoch [80/200], Loss: 0.0000
Epoch [100/200], Loss: 0.0000
Epoch [120/200], Loss: 0.0000
Epoch [140/200], Loss: 0.0000
Epoch [160/200], Loss: 0.0000
Epoch [180/200], Loss: 0.0000
Epoch [200/200], Loss: 0.0000


# **CNN**

In [15]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt


In [16]:
# Transform to Tensor and normalize
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])

# Download training and test datasets
train_dataset = torchvision.datasets.MNIST(root='./data', train=True, transform=transform, download=True)
test_dataset = torchvision.datasets.MNIST(root='./data', train=False, transform=transform)

# Data loaders
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=64, shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=64, shuffle=False)


In [17]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(1, 16, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        self.layer2 = nn.Sequential(
            nn.Conv2d(16, 32, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        self.fc = nn.Linear(32 * 7 * 7, 10)  # 28x28 → 14x14 → 7x7

    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.view(out.size(0), -1)  # Flatten
        out = self.fc(out)
        return out


In [18]:
model = CNN()
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Training loop
num_epochs = 5
for epoch in range(num_epochs):
    for images, labels in train_loader:
        outputs = model(images)
        loss = criterion(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}")


Epoch [1/5], Loss: 0.0064
Epoch [2/5], Loss: 0.0781
Epoch [3/5], Loss: 0.0533
Epoch [4/5], Loss: 0.0117
Epoch [5/5], Loss: 0.1525


# **BI-RNN**

In [19]:
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt


In [20]:
def generate_data(seq_length):
    x = np.linspace(0, 2 * np.pi, seq_length)
    data = np.sin(x)
    return data

sequence_length = 50
data = generate_data(sequence_length + 1)

# Input and target
x = torch.tensor(data[:-1], dtype=torch.float32).reshape(1, sequence_length, 1)  # (batch, seq_len, input_size)
y = torch.tensor(data[1:], dtype=torch.float32).reshape(1, sequence_length, 1)


In [21]:
class BiRNNModel(nn.Module):
    def __init__(self, input_size=1, hidden_size=64, num_layers=1, output_size=1):
        super(BiRNNModel, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers

        # bidirectional=True
        self.rnn = nn.RNN(input_size, hidden_size, num_layers, batch_first=True, bidirectional=True)
        self.fc = nn.Linear(hidden_size * 2, output_size)  # hidden_size * 2 because of both directions

    def forward(self, x):
        h0 = torch.zeros(self.num_layers * 2, x.size(0), self.hidden_size)  # 2 for bidirectional
        out, _ = self.rnn(x, h0)
        out = self.fc(out)
        return out


In [22]:
class BiRNNModel(nn.Module):
    def __init__(self, input_size=1, hidden_size=64, num_layers=1, output_size=1):
        super(BiRNNModel, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers

        # bidirectional=True
        self.rnn = nn.RNN(input_size, hidden_size, num_layers, batch_first=True, bidirectional=True)
        self.fc = nn.Linear(hidden_size * 2, output_size)  # hidden_size * 2 because of both directions

    def forward(self, x):
        h0 = torch.zeros(self.num_layers * 2, x.size(0), self.hidden_size)  # 2 for bidirectional
        out, _ = self.rnn(x, h0)
        out = self.fc(out)
        return out


In [23]:
model = BiRNNModel()
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

epochs = 200
for epoch in range(epochs):
    model.train()
    output = model(x)
    loss = criterion(output, y)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if (epoch + 1) % 20 == 0:
        print(f"Epoch [{epoch+1}/{epochs}], Loss: {loss.item():.4f}")


Epoch [20/200], Loss: 0.0048
Epoch [40/200], Loss: 0.0015
Epoch [60/200], Loss: 0.0003
Epoch [80/200], Loss: 0.0002
Epoch [100/200], Loss: 0.0001
Epoch [120/200], Loss: 0.0001
Epoch [140/200], Loss: 0.0001
Epoch [160/200], Loss: 0.0000
Epoch [180/200], Loss: 0.0000
Epoch [200/200], Loss: 0.0000


# **BPTT**

In [24]:
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt


In [25]:
# Let's predict the next value in this toy sequence
seq = [0.1, 0.2, 0.3, 0.4, 0.5]
x = torch.tensor(seq[:-1], dtype=torch.float32).view(-1, 1)  # Inputs: [0.1, 0.2, 0.3, 0.4]
y = torch.tensor(seq[1:], dtype=torch.float32).view(-1, 1)  # Targets: [0.2, 0.3, 0.4, 0.5]


In [26]:
class RNNCell:
    def __init__(self, input_size, hidden_size):
        self.Wx = torch.randn(input_size, hidden_size, requires_grad=True)
        self.Wh = torch.randn(hidden_size, hidden_size, requires_grad=True)
        self.b = torch.zeros(hidden_size, requires_grad=True)
        self.Wy = torch.randn(hidden_size, 1, requires_grad=True)
        self.by = torch.zeros(1, requires_grad=True)
        self.params = [self.Wx, self.Wh, self.b, self.Wy, self.by]

    def forward(self, x_seq):
        h = torch.zeros(self.Wh.shape[0])
        self.hs = []  # store for BPTT
        self.ys = []

        for x in x_seq:
            h = torch.tanh(x @ self.Wx + h @ self.Wh + self.b)
            y_pred = h @ self.Wy + self.by
            self.hs.append(h)
            self.ys.append(y_pred)
        return self.ys


In [27]:
rnn = RNNCell(input_size=1, hidden_size=10)
lr = 0.01
loss_fn = nn.MSELoss()

for epoch in range(300):
    # Forward
    y_preds = rnn.forward(x)
    loss = sum(loss_fn(y_pred, target) for y_pred, target in zip(y_preds, y))

    # Backward (BPTT)
    for p in rnn.params:
        if p.grad is not None:
            p.grad.zero_()
    loss.backward()

    # Update weights manually
    with torch.no_grad():
        for p in rnn.params:
            p -= lr * p.grad

    if (epoch + 1) % 50 == 0:
        print(f"Epoch {epoch+1}, Loss: {loss.item():.4f}")


Epoch 50, Loss: 0.0001
Epoch 100, Loss: 0.0000
Epoch 150, Loss: 0.0000
Epoch 200, Loss: 0.0000
Epoch 250, Loss: 0.0000
Epoch 300, Loss: 0.0000
